# Sora2 Watermark Remover

AI を使って Sora 動画からウォーターマークを除去します。

**新機能:**
- `--frame-step`: フレーム処理間隔（1=全フレーム、2=隔フレーム）
- `--target-fps`: 出力fps指定（0=入力と同じ）

**リポジトリ:** https://github.com/fulfulggg/Sora2WatermarkRemover


In [ ]:
# リポジトリのクローンとブランチ切替
!git clone https://github.com/fulfulggg/Sora2WatermarkRemover.git
%cd Sora2WatermarkRemover
!git fetch origin --prune
!git checkout main
!git pull --ff-only
print("✅ リポジトリの準備完了")


## ⚠️ 重要: 依存関係の修正

次のセルを実行後、**必ずランタイムを再起動**してください。


In [ ]:
# Pillow 修正（実行後にランタイムを再起動）
!pip uninstall -y pillow PIL
!pip install -U "pillow==10.4.0"
print("\n✅ 完了。ランタイム→ランタイムを再起動 を実行してください。")


In [ ]:
# 再起動後の確認
import PIL
import PIL._util as u

print(f"Pillow: {PIL.__version__}")
assert PIL.__version__ == "10.4.0" and hasattr(u, "is_directory"), "❌ ランタイムを再起動してください"
print("✅ 依存関係OK")


In [ ]:
# 競合除去
!pip uninstall -y diffusers huggingface_hub transformers

# ベースライブラリ（ColabのGPU PyTorchを維持するためtorchは入れない）
!pip install -q "iopaint==1.2.2" opencv-python tqdm loguru

# 安定版ピン（Florence trust_remote_codeとiopaintの両立）
!pip install -q "diffusers==0.26.3" "huggingface_hub==0.25.2" "transformers==4.42.4"

!apt-get -qq install -y ffmpeg

# バージョン確認
from importlib.metadata import version as v
print(f"transformers: {v('transformers')}")
print(f"huggingface_hub: {v('huggingface_hub')}")
print(f"diffusers: {v('diffusers')}")
print(f"iopaint: {v('iopaint')}")
print("✅ パッケージインストール完了")


In [ ]:
# LaMa モデルの事前ダウンロード
!iopaint download --model lama
print("✅ LaMaモデルのダウンロード完了")


In [ ]:
%%bash
set -e
cd /content/Sora2WatermarkRemover
mkdir -p flash_attn
cat > flash_attn/__init__.py <<'PY'
"""
Lightweight stub package for `flash_attn`.
Satisfies import-time checks in Florence-2 (trust_remote_code). Not a real implementation.
"""
def __getattr__(name: str):
    raise RuntimeError(
        f"flash_attn stub in use: attempted to access attribute '{name}'. "
        "This environment runs with eager attention. If you need FlashAttention, "
        "install a matching wheel for your CUDA/SM."
    )
PY
echo "✅ flash_attn stub created at /content/Sora2WatermarkRemover/flash_attn"


In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ GPU未検出。ランタイム→ランタイムのタイプを変更→GPU")


In [ ]:
from google.colab import files

print("動画ファイルをアップロード:")
uploaded = files.upload()
input_video = list(uploaded.keys())[0]
print(f"✅ アップロード: {input_video}")


## パラメータ設定

- `max_bbox_percent`: 検出する最大サイズ（画像比%）
- `frame_step`: 処理間隔（1=全フレーム、2=隔フレーム）
- `target_fps`: 出力fps（0=入力と同じ）


In [ ]:
# パラメータ
max_bbox_percent = 10.0
frame_step = 1
target_fps = 0.0
output_video = "output.mp4"

# 処理実行
import os
import time

input_path = f"/content/{input_video}"
output_path = f"/content/{output_video}"

%cd /content/Sora2WatermarkRemover

print(f"入力: {input_path}")
print(f"出力: {output_path}")
print(f"パラメータ: bbox={max_bbox_percent}%, step={frame_step}, fps={target_fps}")
print("\n🚀 処理開始...\n")

t0 = time.time()
!python remwm.py "{input_path}" "{output_path}" \
  --max-bbox-percent {max_bbox_percent} \
  --frame-step {frame_step} \
  --target-fps {target_fps} \
  --force-format=MP4

elapsed = time.time() - t0

if os.path.exists(output_path):
    size_mb = os.path.getsize(output_path) / (1024 * 1024)
    print(f"\n✅ 処理完了")
    print(f"⏱ 処理時間: {elapsed:.1f}秒")
    print(f"📊 ファイルサイズ: {size_mb:.2f} MB")
else:
    print("\n❌ エラー: 出力ファイルが作成されませんでした")


In [ ]:
# 結果のダウンロード
from google.colab import files

files.download(f"/content/{output_video}")
print("✅ ダウンロード完了")


## 使い方のヒント

### 処理を高速化
```python
frame_step = 2  # 隔フレーム処理
```

### 出力fpsを固定
```python
target_fps = 30
```

### 検出感度を調整
```python
max_bbox_percent = 15  # 大きいロゴも検出
```
